# Enigma A: How Enigma Works

## Create a Enigma with Rotors and Reflector

In [ ]:
# create a function generatea random order of 26 letters
import random
import numpy as np
import pandas as pd


MAX_TRY = 1000_000
DEBUG = False

random.seed(MAX_TRY)

if DEBUG:
    CAPITAL_LETTERS = 'ABCD'
    ROTOR_TOTAL = 1
    plaintext = "abcd"
    init_rotor_position = 'A'
else:
    CAPITAL_LETTERS = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    ROTOR_TOTAL = 3
    init_rotor_position = 'AAA'
    plaintext = "The quick brown fox jumps over the lazy dog."
    

def create_rotors(rotor_total):
    # create a dataframe, use index as index, I as the alphabet column name
    index = list(CAPITAL_LETTERS)
    df_rotors = pd.DataFrame(index=index)
    for roter_i in range(rotor_total):
        try_cnt = 0
        alphabet = list(CAPITAL_LETTERS)
        while try_cnt < MAX_TRY:
            try_cnt += 1
            if try_cnt > MAX_TRY:
                print('Exceed max try')
                break
            else:
                alphabet = list(CAPITAL_LETTERS)
                random.shuffle(alphabet)
                differences = [i!=j for i, j in zip(index, alphabet)]
                if all(differences):
                    break # all True means: no same letter at the same positio, e.g. A->A, B->B
                else:
                    continue
        # create a dictionary to map the original letter to the new letter
        # dict_alphabet = dict(zip(index, alphabet))
        # print(alphabet)
        # convert roter_i into Roman number, e.g. I, II, III, IV, V
        roter_roman = ['I', 'II', 'III', 'IV', 'V'][roter_i]
        df_rotors[roter_roman] = alphabet
        # df_rotors[roter_roman + "_id"] = index
        # df_rotors[roter_roman + "_value"] = alphabet
    return df_rotors  


def create_reflector():
    # Reflector is a special rotor, it does not rotate, it mapps the input to the output, e.g. A->Z, B->Y, C->X, etc.
    # But No self mapping, e.g. A->A, B->B, C->C, etc.
    # create 13 pairs from the 26 letters
    alphabet = list(CAPITAL_LETTERS)
    # random.seed(MAX_TRY) # set the seed for reproducibility
    random.shuffle(alphabet) 
    pairs = [alphabet[i:i+2] for i in range(0, len(alphabet), 2)]
    # convert paris into a dictionary
    pairs = dict(pairs)
    # revise the dictionary to make sure the value of the key is the key of the value
    pairs_revise = {}
    for k, v in pairs.items():
        pairs_revise[v] = k
    # merge the two dictionaries
    pairs.update(pairs_revise)
    # sort the dictionary by the key
    pairs = dict(sorted(pairs.items()))
    # save pairs to a dataframe
    df_reflector = pd.DataFrame(pairs, index=['reflector'])    
    df_reflector = df_reflector.T
    # df_reflector = df_reflector.reset_index(drop=False)
    return df_reflector


def create_rotors_reverse(df_rotors):
    df_temp = df_rotors.copy() 
    df_rotors_reverse = df_rotors.copy() 
    df_temp.reset_index(inplace=True)

    for i in range(ROTOR_TOTAL):
        # select column index and rotor i column
        rotor_name = ['I', 'II', 'III', 'IV', 'V'][i]
        df_temp_pair = df_temp[['index', rotor_name]]

        # sort df_temp_pair by the roter_name column
        df_temp_pair = df_temp_pair.sort_values(by=rotor_name)
        # rename columns 
        df_temp_pair.columns = [rotor_name, 'index']   

        # delete the rotor_name column from df_rotors_reverse
        df_rotors_reverse = df_rotors_reverse.drop(columns=rotor_name)

        # add the column rotor_name to the df_rotors_reverse
        df_rotors_reverse[rotor_name+'_r'] = df_temp_pair[rotor_name].values
        # reverse the order of columns
    df_rotors_reverse = df_rotors_reverse.loc[:, ::-1] 
    return df_rotors_reverse
 
##  Assembleing the rotors, reflector and reverse rotors
# stack the df_rotor and df_rotor_reverse horizontailly
def assemble_engima(df_rotors, df_reflector):
    df_rotors_reverse = create_rotors_reverse(df_rotors)
    df_rotor_reflector_encoder = pd.concat([df_rotors, df_reflector, df_rotors_reverse], axis=1)
    # chent the index name to step  
    df_rotor_reflector_encoder.rename(columns={'index':'name'}, inplace=True)
    return df_rotor_reflector_encoder 


def test_mapping_letter(df_enigma, verbose=False):
    for i, letter in enumerate(CAPITAL_LETTERS):
        # forwad mapping
        letter_mapped = mapping_letter(letter, df_enigma)
        if verbose:
            print(f'forward:\t{letter} -> {letter_mapped}')
        letter2 = letter_mapped
        # backward mapping
        letter_mapped2 = mapping_letter(letter2, df_enigma)
        if verbose:
            print(f'backward:\t{letter2} -> {letter_mapped2}')
        assert letter == letter_mapped2, f'{letter} is not equal to {letter_mapped2}'



# assume the 0 row is the initial position of the rotors
# read the 0 raw all the {I, II, II}_id columns from the df_roters_stepped
def calcualte_rotor_steps(init_rotor_position):
    roters_current_setting_id = list("A"*len(init_rotor_position))
    roters_target_setting_id = list(init_rotor_position)
    roters_steps = [ord(target) - ord(current) for target, current in zip(roters_target_setting_id, roters_current_setting_id)]
    print(f'{roters_current_setting_id} -> {roters_target_setting_id} = {roters_steps}')
    return roters_steps

def test_calcualte_rotor_steps(init_rotor_position = 'AAA'):
    # default df_rotors status is "AAA"
    # from A to Z means 25 steps of change
    roters_steps = calcualte_rotor_steps(init_rotor_position)
    assert roters_steps == [0, 0, 0], f'roters_steps={roters_steps}'

    init_rotor_position = 'BAA' # move one step   
    roters_steps = calcualte_rotor_steps(init_rotor_position)
    assert roters_steps == [1, 0, 0], f'roters_steps={roters_steps}'

    init_rotor_position = 'HIJ' #  
    roters_steps = calcualte_rotor_steps(init_rotor_position)
    assert roters_steps == [7, 8, 9], f'roters_steps={roters_steps}'

    init_rotor_position = 'YBF' #  
    roters_steps = calcualte_rotor_steps(init_rotor_position)
    assert roters_steps == [24, 1, 5], f'roters_steps={roters_steps}'

    init_rotor_position = 'ZZZ'
    roters_steps = calcualte_rotor_steps(init_rotor_position)
    assert roters_steps == [25, 25, 25], f'roters_steps={roters_steps}'

def set_enigma(df_enigma, init_rotor_position = 'AAA'):
    """ set the rotors to the initial position 
    """
    df_enigma_set = df_enigma.copy()
    roters_steps = calcualte_rotor_steps(init_rotor_position)
    
    for i in range(ROTOR_TOTAL):
        roter_roman = ['I', 'II', 'III', 'IV', 'V'][i]
        roter_roman_reverse = roter_roman + '_r'
        for step in range(roters_steps[i]):
            # get the top row, columns [col_rotor_name, col_roter_value]
            rotor_id = df_enigma_set.iloc[0][roter_roman]
            rotor_value = df_enigma_set.iloc[0][roter_roman_reverse]
            col_rotor = [roter_roman, roter_roman_reverse]
            # print(roter_roman, roter_roman, roter_roman_reverse)
            # shift A->B, B->C, C->D, ... Z->A    
            df_enigma_set[col_rotor] = df_enigma_set[col_rotor].shift(-1)
            # backfill the NaN with the 1st row of column [col_rotor_name, roter_roman_reverse]
            df_enigma_set[col_rotor] = df_enigma_set[col_rotor].fillna({roter_roman:rotor_id, roter_roman_reverse:rotor_value})
    return df_enigma_set


def format_input(plaintext):
    input = plaintext
    input = input.upper().replace(' ', '').replace('.', '').replace(',', '').replace('!', '').replace('?', '') 
    # convert into list
    return list(input)

 
def mapping_letter(letter, df_enigma):
    df_enigma = df_enigma.T # transpose the dataframe
    steps_total = df_enigma.shape[0]
    letter_in = letter
    for step in range(steps_total):
        letter_out = df_enigma[letter_in].values[step]
        # print(f'{step= : }    {letter_in} -> {letter_out}')
        letter_in = letter_out # update: out -> in
    return letter_out

def encode_freeze(plaintext, df_engima):
    input = format_input(plaintext)
    df_engima_start = df_engima.copy()
    output = []
    for letter in input:
        output_letter = mapping_letter(letter, df_engima_start)
        output.append(output_letter)
    output = ''.join(output)
    return output

def enigma_stepper(df_engima, roters_steps):    
    df_enigma_next = df_engima.copy()
    for i in range(ROTOR_TOTAL):
        roter_roman = ['I', 'II', 'III', 'IV', 'V'][i]
        roter_roman_reverse = roter_roman + '_r'    
        for step in range(roters_steps[i]): # repeat the steps
            # get the top row, columns [col_rotor_name, col_roter_value]
            rotor_id = df_enigma_next.iloc[0][roter_roman]
            rotor_value = df_enigma_next.iloc[0][roter_roman_reverse]
            col_rotor = [roter_roman, roter_roman_reverse]
            df_enigma_next[col_rotor] = df_enigma_next[col_rotor].shift(-1)
            refill_dict = {roter_roman:rotor_id, roter_roman_reverse:rotor_value} 
            df_enigma_next[col_rotor] = df_enigma_next[col_rotor].fillna(refill_dict)
    return df_enigma_next

def encode_live(plaintext, df_engima):
    input = format_input(plaintext)
    output = []
    base, column = df_engima.shape
    counter = 0
    rotor_cnt = [0, 0, 0]
    for letter in input:
        counter = counter + 1
        rotor_cnt[0] = counter % base
        if rotor_cnt[0] == base - 1:
            rotor_cnt[1] = (rotor_cnt[1] +  1) % base
        if rotor_cnt[1] == base - 1:
            rotor_cnt[2] = (rotor_cnt[2] +  1) % base
        if rotor_cnt[2] == base - 1:
            rotor_cnt[2] = 0
        # rotating the rotors 1 step
        df_engima_next = enigma_stepper(df_engima, rotor_cnt)

        # encode the next letter
        output_letter = mapping_letter(letter, df_engima_next)
        output.append(output_letter) 

        # show output
        # print(f"{letter =} {rotor_cnt = }")
        # display(df_engima_next)
    # list to string
    output = ''.join(output)
    return output, counter, rotor_cnt

# display(df_rotors)
# display(df_reflector)
 
df_rotors = create_rotors(rotor_total=ROTOR_TOTAL)
df_reflector = create_reflector() 
df_enigm_assemble = assemble_engima(df_rotors, df_reflector)
df_enigma = set_enigma(df_enigm_assemble, init_rotor_position)

display(df_enigma)


In [ ]:
test_calcualte_rotor_steps(init_rotor_position=init_rotor_position)
test_mapping_letter(df_enigma) 
 

In [ ]:
# encrypt the plaintext into ciphertext
for i in range(2):
    ciphertext = encode_freeze(plaintext, df_enigma)
    message = encode_freeze(ciphertext, df_enigma)
    # 
    print(f'{"="*25} Round {i} {"="*25}')
    print(f'{plaintext  =}')
    print(f'{ciphertext =}')
    print(f'{message    =}')

for i in range(2):
    ciphertext, ciphertext_cnt, ciphertext_rotor_cnt = encode_live(plaintext, df_enigma)
    message, message_cnt, message_rotor_cnt = encode_live(ciphertext, df_enigma)    # 
    print(f'{"="*25} Round {i} {"="*25}')
    # input = format_input(plaintext)
    # input = ''.join(input)
    # print(f'{input      =} {len(input)}')
    print(f'{plaintext  =}')
    print(f'{ciphertext =} {ciphertext_cnt}:{ciphertext_rotor_cnt}' )
    print(f'{message    =} {message_cnt}:{message_rotor_cnt}')



## Save the Engima

In [ ]:
# Save the csv file
engima_file = 'engima.csv'
df_enigma.to_csv(engima_file, index=True, index_label='index')
# df_rotor_reflector_encoder

##  Simple PlainText Test

In [ ]:
from pathlib import Path

# load the engima_file file if it exists
if Path(engima_file).exists():
    df_enigma = pd.read_csv(engima_file, index_col=0)
    # display(df_enigma)
else:
    print(f'{engima_file} does not exist')
    # create a new engima file


In [ ]:
# encrypt the plaintext into ciphertext
ciphertext = encode_freeze(plaintext, df_enigma)

print(f'{plaintext  =}')
print(f'{ciphertext =}')

# decrypt the ciphertext into message by just put the ciphertext into the encode function again.
message = encode_freeze(ciphertext, df_enigma)
print(f'{message    =}')